# OuterSPACE

## Imports

Import the necessary modules.

In [ ]:
# HiFiber boilerplate

from fibertree_bootstrap import *

fibertree_bootstrap(style="tree", animation='movie')

# Compilation boilerplate

import os
import sys
sys.path.insert(0, "..")

from src.utils import *

## Initialization

Initialize the input tensors. Tensor shapes and densities can be modified below.

**Warning:** Large tensors will overwhelm the video generation. Either:
1. Use small tensors; as a rule of thumb, fewer than 60 computes (e.g., multiplications) should be required.
2. Do not generate a video; remove the `spacetime` specification from the `mapping` before compiling.

In [ ]:
K = 4
M = 5
N = 6

density = [0.9, 0.5]
seed = 0

A_KM = Tensor.fromRandom(rank_ids=["K", "M"], shape=[K, M], seed=seed, density=density, name="A")
B_KN = Tensor.fromRandom(rank_ids=["K", "N"], shape=[K, N], seed=seed + 1, density=density, name="B")

Note: Small tensors are required for video generation. Partition shapes are decreased for visualization purposes.

## Compile and Run OuterSPACE

Below is the TeAAL specification for OuterSpace. To simulate OuterSPACE:
1. Compile it to HiFiber by running the cell, inserting a new cell
2. Run the new cell, which will
    - Execute the kernel; multiplying the above defined matrices
    - Generate visualizations of the actions of the kernel

Remember, if you are using large tensors, remove the spacetime specification to generate a kernel that does not produce videos. Outputs can still be checked below.

In [ ]:
yaml = """
einsum:
    declaration:
        A: [K, M]
        B: [K, N]
        T: [K, M, N]
        Z: [M, N]
    expressions:
        - T[k, m, n] = A[k, m] * B[k, n]
        - Z[m, n] = T[k, m, n]
mapping:
    rank−order:
        A: [K, M]
        B: [K, N]
        T: [M, K, N]
        Z: [M, N]
    partitioning:
        T:
            (K, M): [flatten()]
            KM: [uniform_occupancy(A.4), uniform_occupancy(A.2)]
        Z:
            M: [uniform_occupancy(T.4), uniform_occupancy(T.2)]
    loop−order:
        T: [KM2, KM1, KM0, N]
        Z: [M2, M1, M0, N, K]
    spacetime:
        T:
            space: [KM1, KM0]
            time: [KM2, N]
        Z:
            space: [M1, M0]
            time: [M2, N, K]
"""

compile(yaml)

## Check Results

Check that generated code computes the correct result.

**Note**: Should be used after compiling and running the kernel (above cell)

In [ ]:
check_matmul(A_KM, B_KN, Z_MN)